# From Datetime to MEM format

Take normal datetime format of the pandas equivalent and covert
the file to MEM style: year, month, day, hour format


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import csv
import matplotlib.pyplot as plt

In [5]:
def make_MEM_compatible_NORMAL(f_name, save_name, date_time_field='date_time'):
    print(f"In file: {f_name}")
    df = pd.read_csv(f_name)
    df['date_time'] = pd.to_datetime(df[date_time_field])
    
    with open(f'{save_name}.csv', 'w', newline='') as csvfile:

        fieldnames = ['year', 'month', 'day', 'hour', 'demand (MW)', 'date_time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for idx in df.index:
            # This strange shift is to uses MEM's 1-24 counting,
            # see example datetime and MEM for these three hours:
            # year,month,day,hour,date_time
            # 2015,7,2,23,2015-07-02 23:00:00
            # 2015,7,2,24,2015-07-03 00:00:00
            # 2015,7,3,1,2015-07-03 01:00:00
            mem_format = df.loc[idx, 'date_time'] + timedelta(hours=-1)
            writer.writerow({
                'year': mem_format.year,
                'month': mem_format.month,
                'day': mem_format.day,
                'hour': mem_format.hour+1,
                'demand (MW)': int(round(df.loc[idx, 'estimated_load'])),
                'date_time': df.loc[idx, 'date_time']
            })
    print(f"Outfile: {save_name}")

def make_MEM_compatible_DAVES(f_name, save_name, date_time_field='date_time'):
    print(f"In file: {f_name}")
    df = pd.read_csv(f_name)
    df['date_time'] = pd.to_datetime(df[date_time_field])
    
    with open(f'{save_name}.csv', 'w', newline='') as csvfile:

        # Write BEGIN_DATA before header
        writer = csv.writer(csvfile)
        writer.writerow(['BEGIN_DATA'])
        
        fieldnames = ['year', 'month', 'day', 'hour', 'demand (MW)',]# 'date_time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        # Write one line that only says BEGIN_DATA before header
        writer.writeheader()
        
        for idx in df.index:
            # This strange shift is to uses MEM's 1-24 counting,
            # see example datetime and MEM for these three hours:
            # year,month,day,hour,date_time
            # 2015,7,2,23,2015-07-02 23:00:00
            # 2015,7,2,24,2015-07-03 00:00:00
            # 2015,7,3,1,2015-07-03 01:00:00
            mem_format = df.loc[idx, 'date_time']
            writer.writerow({
                'year': mem_format.year,
                'month': mem_format.month,
                'day': mem_format.day,
                'hour': mem_format.hour+1,
                'demand (MW)': int(round(df.loc[idx, 'cleaned demand (MW)'])), # demand was in kW
                #'date_time': df.loc[idx, 'date_time']
            })
    print(f"Outfile: {save_name}")

In [6]:
base = './'

for region in ['CISO', 'ERCO', 'ISNE', 'MISO']:
    make_MEM_compatible_DAVES(base+'{0}.csv'.format(region), '{0}_demand_reformatted'.format(region))

In file: ./CISO.csv
Outfile: CISO_demand_reformatted
In file: ./ERCO.csv
Outfile: ERCO_demand_reformatted
In file: ./ISNE.csv
Outfile: ISNE_demand_reformatted
In file: ./MISO.csv
Outfile: MISO_demand_reformatted
